In [1]:
import os
import yaml
import json
import pandas as pd
from pathlib import Path

In [2]:
# parser.add_argument('mode', type=str,
#     choices=['make-ome', 'segment-ome', 'generate-spatial-features', 'show-channels'],
#     help='Which task mip is to execute.')

# ##############
# ## make-ome ##
# ##############
# parser.add_argument('--input-tif', type=str,
#     help='Used in make-ome mode. Either directory of stitched tif files that will be combined into a single ome.tiff file, a multichannel .tif (for original codex platform), or a .qptiff (phenocycler platform).')

# parser.add_argument('--output-filepath', type=str,
#     help='Location to write ome.tiff file')

# parser.add_argument('--platform', type=str,
#     choices=['codex', 'phenocycler', 'raw'], default='phenocycler',
#     help='Which platform produced the input images. phenocycler assumes a .qptiff from the akoya phenocycler platform. codex assumes a multichannel .tif output by the original akoya codex platform. raw will save a directory of .tifs together into a multiplex image.')

# parser.add_argument('--bbox', type=str,
#     help='If desired, bbox in to crop image with. Must be the following format: "top,bottom,left,right"')

In [3]:
cwl_dir = '/diskmnt/Projects/Users/estorrs/multiplex-imaging-pipeline/cwl'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

In [4]:
template = {
    'bbox': 'a_string',
    'platform': 'a_string',
    'output_filepath': 'a_string',
    'input_tif': {
        'class': 'File',
        'path': 'a/file/path',
    },
}

In [5]:
yaml.safe_dump(template, open(os.path.join(cwl_dir, 'template.ome_generation.yaml'), 'w'))

In [6]:
cwl = {
    'class': 'CommandLineTool',
    'cwlVersion': 'v1.0',
    '$namespaces': {'sbg': 'https://www.sevenbridges.com/'},
    'id': 'ome_generation',
    'label': 'ome_generation',
    'baseCommand': ['mip'],
    'inputs': [
        {
            'id': 'input_tif',
            'type': 'File',
            'inputBinding': {
                'position': '0',
                'prefix': '--input-tif',
            }
        },
        {
            'id': 'output_filepath',
            'type': 'string?',
            'default': 'output.ome.tiff',
            'inputBinding': {
                'prefix': '--output-filepath',
                'position': '0'
            }
        },
        {
            'id': 'platform',
            'type': 'string?',
            'default': 'phenocycler',
            'inputBinding': {
                'prefix': '--platform',
                'position': '0'
            }
        },
        {
            'id': 'bbox',
            'type': 'string?',
            'inputBinding': {
                'prefix': '--bbox',
                'position': '0'
            }
        },
        
        # needs path to be set so it works on compute1
        {
            'id': 'environ_PATH',
            'type': 'string?',
            'default': '/miniconda/envs/mip/bin:$PATH'
        },
    ],
    'arguments': [
        {
            'position': 1,
            'valueFrom': 'make-ome'
        },
    ],
    'outputs': [
        {
            'id': 'output_ome_tiff',
            'type': 'File',
            'outputBinding': {'glob': '*.ome.tiff'}
        },
    ],
    'requirements': [
        {
            'class': 'DockerRequirement',
            'dockerPull': 'estorrs/multiplex-imaging-pipeline:0.0.1'
        },
        {
            'class': 'ResourceRequirement',
            'ramMin': 60000
        },
        {
            'class': 'EnvVarRequirement',
            'envDef': {
                'PATH': '$(inputs.environ_PATH)',
            }
        }
    ]
}

In [7]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'ome_generation.cwl'), 'w'))